In [1]:
import cirq
import numpy as np
import sympy

In [2]:
def n_qubit_initialise(n):
    """We use this function to initialise n qubits on which we can perform a generalised QFT."""
    qubits = cirq.LineQubit.range(n)
    return qubits

In [3]:
class CZpow(cirq.Gate):
    def __init__(self, d):
        self.d = d

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array(
            [
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, (1j)**(1/(2**(self.d - 1)))],
            ]
        )

    def _circuit_diagram_info_(self, args):
        return '@', f'CZ({self.d})'

In [4]:
def qft_moments(n):
    ops_1 = []
    ops_2 = []
    for i in range(n):
        ops_1 = [cirq.H(qubits[i])]
        ops_2 = [CZpow(j-i)(qubits[j],qubits[i]) for j in np.arange(i+1,n)]
        circuit_qft.append([ops_1,ops_2], strategy = cirq.InsertStrategy.NEW)

In [5]:
qubits = n_qubit_initialise(2)
circuit_qft = cirq.Circuit()

In [6]:
qft_moments(2)
print(circuit_qft)

0: ───H───CZ(1)───────
          │
1: ───────@───────H───


In [7]:
simulator = cirq.Simulator()
result = simulator.simulate(circuit_qft, initial_state=(1,0))
print(result)

measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: 0.5|00⟩ + 0.5|01⟩ - 0.5|10⟩ - 0.5|11⟩

phase:
output vector: |⟩


If the input qubits are encoded in reverse, then the output is coded correctly i.e least significant qubits from the top. The QFT is a subroutine for the quantum phase estimation algorithm.

In [16]:
cirq.unitary(circuit_qft)

array([[ 0.5+0.j ,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ],
       [ 0.5+0.j , -0.5+0.j ,  0.5+0.j , -0.5+0.j ],
       [ 0.5+0.j ,  0. +0.5j, -0.5+0.j ,  0. -0.5j],
       [ 0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.5j]])

In [8]:
qft_op = cirq.qft(*qubits, without_reverse=True)
circuit_inbuilt = cirq.Circuit(qft_op)
cirq.unitary(circuit_inbuilt)


array([[ 0.5+0.j ,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ],
       [ 0.5+0.j , -0.5+0.j ,  0.5+0.j , -0.5+0.j ],
       [ 0.5+0.j ,  0. +0.5j, -0.5+0.j ,  0. -0.5j],
       [ 0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.5j]])

The phase estimation has three layers followed by a measurement. The first layer is the n-qubit Hadamard on the n-register that stores the value of the eigenvalue $\theta = y/2^n$. The second layer involves applications of control-U on the eigenstate $\Psi$ prepared in the second register with exponentially increasing powers with control on the first register qubits. The third layer is the n-qubit inverse QFT.

In [9]:
# Function for applying control U operations, assuming that the target is a single qubit state.
qubits = n_qubit_initialise(4)
m = cirq.NamedQubit('m')
circuit_qpe = cirq.Circuit()
def controlledUops(n, theta):
    for i in range(n):
        circuit_qpe.append(cirq.CZPowGate(exponent=2*theta*(2**(n-1-i)))(qubits[i],m))
theta = sympy.Symbol('theta')
circuit_qpe.append([cirq.H(qubits[i]) for i in range(4)])
controlledUops(4, theta)
third_op = cirq.qft(*qubits, without_reverse=False, inverse=True)
circuit_qpe.append(third_op)
circuit_qpe.append(cirq.measure(*qubits))
print(circuit_qpe)

0: ───H───@────────────────────────────────────────────────────────qft^-1───M───
          │                                                        │        │
1: ───H───┼──────────────@─────────────────────────────────────────#2───────M───
          │              │                                         │        │
2: ───H───┼──────────────┼─────────────@───────────────────────────#3───────M───
          │              │             │                           │        │
3: ───H───┼──────────────┼─────────────┼─────────────@─────────────#4───────M───
          │              │             │             │
m: ───────@^(16*theta)───@^(8*theta)───@^(4*theta)───@^(2*theta)────────────────


In [10]:
simulator = cirq.Simulator()
results = simulator.simulate_sweep(circuit_qpe, params=[{"theta": i/16} for i in range(16)], qubit_order=(*qubits, m), initial_state=(0, 0, 0, 0, 1))

In [11]:
for result in results:
    print(f"param: {result.params}, result: {result}\n")

param: cirq.ParamResolver({'theta': 0.0}), result: measurements: q(0),q(1),q(2),q(3)=0000

qubits: (cirq.LineQubit(0),)
output vector: |0⟩

qubits: (cirq.LineQubit(1),)
output vector: |0⟩

qubits: (cirq.LineQubit(2),)
output vector: |0⟩

qubits: (cirq.LineQubit(3),)
output vector: |0⟩

qubits: (cirq.NamedQubit('m'),)
output vector: |1⟩

phase:
output vector: |⟩

param: cirq.ParamResolver({'theta': 0.0625}), result: measurements: q(0),q(1),q(2),q(3)=0001

qubits: (cirq.LineQubit(0),)
output vector: |0⟩

qubits: (cirq.LineQubit(1),)
output vector: |0⟩

qubits: (cirq.LineQubit(2),)
output vector: |0⟩

qubits: (cirq.LineQubit(3),)
output vector: |1⟩

qubits: (cirq.NamedQubit('m'),)
output vector: |1⟩

phase:
output vector: |⟩

param: cirq.ParamResolver({'theta': 0.125}), result: measurements: q(0),q(1),q(2),q(3)=0010

qubits: (cirq.LineQubit(0),)
output vector: |0⟩

qubits: (cirq.LineQubit(1),)
output vector: |0⟩

qubits: (cirq.LineQubit(2),)
output vector: |1⟩

qubits: (cirq.LineQubit(3),